<a href="https://colab.research.google.com/github/sololzano/2021-Python-Optimization-Lab/blob/main/W8_Genetic_Algorithm_%2B_BoxPlots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Genetic Algorithm

In [ ]:
# Import libraries
from matplotlib import pyplot as plt
import numpy as np
import time

In [ ]:
# Six Hump Camel Function
def binary_camel(x, encoding, x_min):
  # From binary encoding to real number
  new_x = np.array([[int(c, 2) for c in y] for y in x])
  X = (new_x * encoding) + x_min
  x1, x2 = X[:, 0], X[:, 1]
  
  # Same
  a = (4 - 2.1*x1**2 + (1/3)*x1**4)*x1**2
  b = x1*x2
  c = (-4 + 4*x2**2)*x2**2
  return a + b +c

In [ ]:
# Adjiman's Benchmark Function 
def adjiman(x, encoding, x_min):
  # From binary encoding to real number
  new_x = np.array([[int(c, 2) for c in y] for y in x])
  X = (new_x * encoding) + x_min
  x1, x2 = X[:, 0], X[:, 1]
  a = np.cos(x1) * np.sin(x2)
  b = x1 / (x2**2 + 1)
  return (a - b)

In [ ]:
def init_population(fitness_func, pop_size, dimensions, precision, x_min, x_max):
  x_decimal = np.random.randint(0, (2**precision) - 1, 
                                (pop_size, dimensions))
  # From base 10 to binary
  x_binary = [[format(j, '0{}b'.format(precision)) for j in i] for i in x_decimal]
  
  encoding = (x_max - x_min)/((2**precision) - 1)

  # Calculate fitness
  fitness = fitness_func(x_binary, encoding, x_min)

  # Get best 
  min_idx = np.argmin(fitness)
  gb = x_binary[min_idx]
  fgb = fitness[min_idx]

  return np.array(x_binary), np.array(fitness), np.array(gb), fgb, encoding

In [ ]:
def selection_mechanism(fitness, kind):
  if kind == 'random':
    indices = np.random.permutation(len(fitness))[:2]
    return indices
  if kind == 'tournament':
    indices = np.random.permutation(len(fitness))[:4]
    f_indices = fitness[indices]
    idx1 = np.argmin(f_indices[0:2])
    idx2 = np.argmin(f_indices[2:]) + 2
    indices = [indices[idx1], indices[idx2]]
    return np.array(indices)
  if kind == 'roulette':
    ranks = np.argsort(fitness)
    ranks_vals = 1/(ranks + 1)
    probs = ranks_vals/np.sum(ranks_vals)
    idx1 = np.random.choice(ranks, p=probs)
    idx2 = np.random.choice(ranks, p=probs)
    indices = [idx1, idx2]
    return np.array(indices)

In [ ]:
def crossover(parents, pc, kind, n=0):
  p = np.random.uniform()
  if p > pc:
    return parents

  if kind == 'single_point':
    l = len(parents[0][0])
    idx = np.random.randint(l//4, (3*l)//4)
    p1 = parents[0]
    p2 = parents[1]
    c1 = [p1[0][:idx] + p2[0][idx:], p1[1][:idx] + p2[1][idx:]]
    c2 = [p2[1][idx:] + p1[1][:idx], p2[1][:idx] + p1[1][idx:]]
    children = [c1, c2]
    children = np.array(children)
  elif kind == 'two_point':
    children = np.copy(parents)
    for _ in range(2):
      children = crossover(children, pc, 'single_point')
  return children

In [ ]:
def mutation(child, pm, kind):
  if kind == 'bit_flip':
    new_child = []
    for i in range(len(child)):
      s = ''
      for j in range(len(child[i])):
        p = np.random.uniform()
        if p > pm:
          if child[i][j] == '1':
            s += '0'
          else:
            s += '1'
        else:
          s += child[i][j]
      new_child.append(s)
  return np.array(new_child)

In [ ]:
def maintenance_mechanism(parents, children, parents_fitness, 
                          children_fitness, kind):
  if kind == 'replacement':
    return children, children_fitness
  if kind == 'fittest':
    pooled_population = np.concatenate((parents, children), axis=0)
    pooled_fitness = np.concatenate((parents_fitness, children_fitness))
    indices = np.argsort(pooled_fitness)[:len(pooled_fitness)//2]
    new_pop = pooled_population[indices]
    new_fit = pooled_fitness[indices]
    return new_pop, new_fit
  if kind == 'tournament':
    return 0

In [ ]:
def genetic_algorithm(fitness_function, pop_size, dimensions, precision, 
                      x_min, x_max, max_generations, selection_kind, 
                      crossover_kind, mutation_kind, maintenance_kind, pc, pm):
  
  # Initial population
  x, fx, gb, fgb, encoding = init_population(fitness_function, pop_size, dimensions, 
                                        precision, x_min, x_max)
  gb_array = []
  fb_array = []
  gb_array.append(gb)
  for i in range(1, max_generations + 1):
    new_generation = []
    new_fitness = []
    for j in range(pop_size // 2):
      # Get two potential parents
      idx_parents = selection_mechanism(fx, selection_kind)
      parents = np.copy(x[idx_parents])

      # Cross parents to generate two children
      children = crossover(parents, pc, crossover_kind)
      # Mutate children
      for k in range(len(children)):
        children[k] = mutation(children[k], pm, mutation_kind)
      new_generation.append(np.array(children))

      # Get fitness of children
      new_fitness.append(fitness_function(children, encoding, x_min))

    # Re-shape new generation and respective fitness
    new_generation = np.reshape(new_generation, x.shape)
    new_fitness = np.reshape(new_fitness, fx.shape)

    # Maintenance mechanism
    x, fx = maintenance_mechanism(x, new_generation, fx, 
                                  new_fitness, maintenance_kind)
    min_idx = np.argmin(fx)
    gb_array.append(x[min_idx])
    fb_array.append(fx[min_idx])
  return gb_array, fb_array

In [ ]:
gb_array, fb_array = genetic_algorithm(binary_camel, pop_size=20, dimensions=2, 
                                       precision=25, x_min=-3, x_max=3, 
                                       max_generations=100, 
                                       selection_kind='tournament', 
                                       crossover_kind='single_point', 
                                       mutation_kind='bit_flip', 
                                       maintenance_kind='fittest', 
                                       pc=0.8, pm=0.8)
plt.figure(figsize=(12, 6))
plt.plot(fb_array, linewidth=2)
plt.title('Minimum value {:,.5f}'.format(fb_array[-1]), fontsize=18)
plt.grid()

# Boxplot

In [ ]:
fbs_fittest = []
fbs_replace = []
for _ in range(20):
  _, fb_array = genetic_algorithm(binary_camel, pop_size=20, dimensions=2, 
                                        precision=25, x_min=-3, x_max=3, 
                                        max_generations=50, 
                                        selection_kind='roulette', 
                                        crossover_kind='single_point', 
                                        mutation_kind='bit_flip', 
                                        maintenance_kind='fittest', 
                                        pc=0.5, pm=0.5)
  fbs_fittest.append(fb_array[-1])
for _ in range(20):
  _, fb_array = genetic_algorithm(binary_camel, pop_size=20, dimensions=2, 
                                        precision=25, x_min=-3, x_max=3, 
                                        max_generations=50, 
                                        selection_kind='roulette', 
                                        crossover_kind='single_point', 
                                        mutation_kind='bit_flip', 
                                        maintenance_kind='replacement', 
                                        pc=0.5, pm=0.5)
  fbs_replace.append(fb_array[-1])

In [ ]:
plt.figure(figsize=(6, 10))
ax1 = plt.boxplot([fbs_fittest, fbs_replace], 
                  labels=['Fittest', 'Replacement'])
plt.xticks(fontsize=16)
plt.grid()